In [1]:
from google.colab import drive
drive.mount('/content/drive')

# %cd "drive/MyDrive/DL for Genomics Final Project/Data Demos/GC MERGE Data"
%cd "drive/Shareddrives/DeepBio"
%ls

Mounted at /content/drive
/content/drive/Shareddrives/DeepBio
 checkpoints/  'Full Model.ipynb'   GM12878/  'HiC Visualization'
 E116/         'GC Merge Data'/     H1-hESC/


In [2]:
import numpy as np
from scipy.sparse import load_npz, find
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, Flatten, Dense, MaxPooling1D, Reshape, Conv2DTranspose
import tensorflow_probability as tfp

In [3]:
base_path = "/content/drive/MyDrive/GC Merge Data/data"
cell_line = "E116"
regression_flag = 0
chip_res = 10000
hic_res = 10000
num_hm = 5
num_feat = int((hic_res/chip_res)*num_hm)
slice_len = 256

In [4]:
def downsample_sparse_hic_matrix(sparse_matrix, downsample_factor):
    # Generate random scaling factors for each read, ensuring a different factor for each
    random_factors = tf.random.uniform(tf.shape(sparse_matrix.values), 0, downsample_factor, dtype=tf.dtypes.double)

    # Apply downsampling
    downsampled_values = sparse_matrix.values * random_factors
    downsampled_sparse_tensor = tf.SparseTensor(indices=sparse_matrix.indices, values=downsampled_values, dense_shape=sparse_matrix.dense_shape)

    # Normalize the downsampling to ensure the total reads is approximately 1/16th of the original
    total_reads_original = tf.reduce_sum(sparse_matrix.values)
    total_reads_downsampled = tf.reduce_sum(downsampled_sparse_tensor.values)
    adjustment_factor = (total_reads_original / 16) / total_reads_downsampled
    adjusted_values = downsampled_sparse_tensor.values * adjustment_factor
    adjusted_downsampled_sparse_tensor = tf.SparseTensor(indices=sparse_matrix.indices, values=adjusted_values, dense_shape=sparse_matrix.dense_shape)

    return adjusted_downsampled_sparse_tensor

In [5]:
class Autoencoder(tf.keras.Model):
  def __init__(self, hic_encoder, epigenomic_encoder, decoder):
    super().__init__()
    self.hic_encoder = hic_encoder
    self.epigenomic_encoder = epigenomic_encoder
    self.decoder = decoder

  def call(self, inputs, *args, **kwargs):
    low_hic, hms = inputs
    hic_encoded = self.hic_encoder(low_hic, *args, **kwargs)
    hm_encoded = self.epigenomic_encoder(hms, *args, **kwargs)

    concat_encodings = tf.concat((hic_encoded, hm_encoded), axis=1)
    # print(f"Combined Multimodal Shape: {concat_encodings.shape}")
    recon_hic = self.decoder(concat_encodings,*args, **kwargs)

    return recon_hic

hic_encoder = Sequential(
    layers = [tf.keras.layers.InputLayer(input_shape=(slice_len, slice_len, 1)),
              Conv2D(1, 16, (3, 3), activation='relu'),
              MaxPooling2D((2, 2)),
              Conv2D(16, (3, 3), activation='relu'),
              MaxPooling2D((2, 2)),
              Conv2D(64, (3, 3), activation='relu'),
              MaxPooling2D((2, 2)),
              Flatten()],
    name="Convoltuional HiC Slice Encoder")

epigenomic_encoder = Sequential(
    layers = [tf.keras.layers.InputLayer(input_shape=(slice_len,num_feat)),
              Conv1D(16, 3, activation='relu'),
              MaxPooling1D(2),
              Conv1D(32, 3, activation='relu'),
              MaxPooling1D(2),
              Conv1D(64, 3, activation='relu'),
              MaxPooling1D(2),
              Conv1D(128, 3, activation='relu'),
              MaxPooling1D(2),
              Conv1D(128, 3, activation='relu'),
              MaxPooling1D(2),
              Flatten()],
    name="1D Convoltuional Epgigenomic Encoder")

decoder = Sequential([tf.keras.layers.InputLayer(input_shape=(4864)),
                      Dense(1024),
                      Reshape(target_shape=(32, 32, 1)),
                      Conv2DTranspose(4, (3,3), strides=2, activation="relu", padding="same"),
                      Conv2DTranspose(32, (3,3), strides=2, activation="relu", padding="same"),
                      Conv2DTranspose(64, (3,3), strides=2, activation="relu", padding="same"),
                      Conv2D(1, (3, 3), activation='relu', padding="same"),
                      ], name="HiC Decoder")

autoencoder = Autoencoder(hic_encoder, epigenomic_encoder, decoder)
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(), loss = tf.keras.losses.MeanSquaredError())

In [6]:
# Initialize the model
discriminator = Sequential([tf.keras.layers.InputLayer(input_shape=(slice_len, slice_len, 1)),
                            Conv2D(16, (3, 3), activation='relu'),
                            MaxPooling2D((2, 2)),
                            Conv2D(32, (3, 3), activation='relu'),
                            MaxPooling2D((2, 2)),
                            Conv2D(32, (3, 3), activation='relu'),
                            MaxPooling2D((2, 2)),
                            Conv2D(64, (3, 3), activation='relu'),
                            MaxPooling2D((2, 2)),
                            Conv2D(128, (3, 3), activation='relu'),
                            MaxPooling2D((2, 2)),
                            Flatten(),
                            Dense(512, activation='relu'),
                            Dense(64, activation='relu'),
                            Dense(1, activation='sigmoid')],
                           name="Discriminator")

# Compile the model
discriminator.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
discriminator.summary()

Model: "Discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 16)      160       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 127, 127, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 32)      

In [7]:
autoencoder(inputs = (tf.zeros((1, 256, 256)), tf.zeros((1, 256, 5))), training=False)
discriminator(tf.zeros((1, 256, 256, 1)), training=False)

chpt_dir = f"./checkpoints/full_train scuff"
autoencoder.load_weights(f"{chpt_dir}/autoencoder_weights.keras")
decoder.load_weights(f"{chpt_dir}/decoder_weights.keras")

In [28]:
chr_num = 11
print(f"Testing on Chromosome {chr_num}")
folder = f"{base_path}/{cell_line}/{hic_res}/chr{chr_num}"
hic_sparse_mat_file = f"{folder}/hic_sparse_redone_h3k27ac_addition.npz"

np_hmods_norm_all_file = f"{folder}/np_hmods_norm_chip_{chip_res}bp_redone_h3k27ac_addition.npy"

mat = load_npz(hic_sparse_mat_file)

hms = np.load(np_hmods_norm_all_file)
hms = hms[:, 1:] #only includes features, not node ids
hms = tf.convert_to_tensor(hms.reshape(-1, num_feat), dtype=tf.float32)

Testing on Chromosome 11


In [9]:
## TAKEN FROM CAESAR!! https://github.com/liu-bioinfo-lab/caesar
import os
import numpy as np
# from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec


def visualize_HiC_epigenetics(HiC, epis, output, fig_width=12.0,
                              vmin=0, vmax=None, cmap='Reds', colorbar=True,
                              colorbar_orientation='vertical',
                              epi_labels=None, x_ticks=None, fontsize=24,
                              epi_colors=None, epi_yaxis=True,
                              heatmap_ratio=0.6, epi_ratio=0.1,
                              interval_after_heatmap=0.05, interval_between_epi=0.01, ):
    """
    Visualize matched HiC and epigenetic signals in one figure
    Args:
        HiC (numpy.array): Hi-C contact map, only upper triangle is used.
        epis (list): epigenetic signals
        output (str): the output path. Must in a proper format (e.g., 'png', 'pdf', 'svg', ...).
        fig_width (float): the width of the figure. Then the height will be automatically calculated. Default: 12.0
        vmin (float): min value of the colormap. Default: 0
        vmax (float): max value of the colormap. Will use the max value in Hi-C data if not specified.
        cmap (str or plt.cm): which colormap to use. Default: 'Reds'
        colorbar (bool): whether to add colorbar for the heatmap. Default: True
        colorbar_orientation (str): "horizontal" or "vertical". Default: "vertical"
        epi_labels (list): the names of epigenetic marks. If None, there will be no labels at y axis.
        x_ticks (list): a list of strings. Will be added at the bottom. THE FIRST TICK WILL BE AT THE START OF THE SIGNAL, THE LAST TICK WILL BE AT THE END.
        fontsize (int): font size. Default: 24
        epi_colors (list): colors of epigenetic signals
        epi_yaxis (bool): whether add y-axis to epigenetic signals. Default: True
        heatmap_ratio (float): the ratio of (heatmap height) and (figure width). Default: 0.6
        epi_ratio (float): the ratio of (1D epi signal height) and (figure width). Default: 0.1
        interval_after_heatmap (float): the ratio of (interval between heatmap and 1D signals) and (figure width). Default: 0.05
        interval_between_epi (float): the ratio of (interval between 1D signals) and (figure width). Default: 0.01

    No return. Save a figure only.
    """

    # Make sure the lengths match
    # len_epis = [len(epi) for epi in epis]
    # if max(len_epis) != min(len_epis) or max(len_epis) != len(HiC):
    #     raise ValueError('Size not matched!')
    N = len(HiC)

    # Define the space for each row (heatmap - interval - signal - interval - signal ...)
    rs = [heatmap_ratio, interval_after_heatmap] + [epi_ratio, interval_between_epi] * len(epis)
    rs = np.array(rs[:-1])

    # Calculate figure height
    fig_height = fig_width * np.sum(rs)
    rs = rs / np.sum(rs)  # normalize to 1 (ratios)
    fig = plt.figure(figsize=(fig_width, fig_height))

    # Split the figure into rows with different heights
    gs = GridSpec(len(rs), 1, height_ratios=rs)

    # Ready for plotting heatmap
    ax0 = plt.subplot(gs[0, :])
    # Define the rotated axes and coordinates
    coordinate = np.array([[[(x + y) / 2, y - x] for y in range(N + 1)] for x in range(N + 1)])
    X, Y = coordinate[:, :, 0], coordinate[:, :, 1]
    # Plot the heatmap
    vmax = vmax if vmax is not None else np.max(HiC)
    im = ax0.pcolormesh(X, Y, HiC, vmin=vmin, vmax=vmax, cmap=cmap)
    ax0.axis('off')
    ax0.set_ylim([0, N])
    ax0.set_xlim([0, N])
    if colorbar:
        if colorbar_orientation == 'horizontal':
            _left, _width, _bottom, _height = 0.12, 0.25, 1 - rs[0] * 0.25, rs[0] * 0.03
        elif colorbar_orientation == 'vertical':
            _left, _width, _bottom, _height = 0.9, 0.02, 1 - rs[0] * 0.7, rs[0] * 0.5
        else:
            raise ValueError('Wrong orientation!')
        cbar = plt.colorbar(im, cax=fig.add_axes([_left, _bottom, _width, _height]),
                            orientation=colorbar_orientation)
        cbar.ax.tick_params(labelsize=fontsize)
        cbar.outline.set_visible(False)

    # print(rs/np.sum(rs))
    # Ready for plotting 1D signals
    if epi_labels:
        assert len(epis) == len(epi_labels)
    if epi_colors:
        assert len(epis) == len(epi_colors)

    for i, epi in enumerate(epis):
        # print(epi.shape)
        ax1 = plt.subplot(gs[2 + 2 * i, :])

        if epi_colors:
            ax1.fill_between(np.arange(N), 0, epi, color=epi_colors[i])
        else:
            ax1.fill_between(np.arange(N), 0, epi)
        ax1.spines['left'].set_visible(False)
        ax1.spines['right'].set_visible(False)
        ax1.spines['top'].set_visible(False)
        ax1.spines['bottom'].set_visible(False)

        if not epi_yaxis:
            ax1.set_yticks([])
            ax1.set_yticklabels([])
        else:
            ax1.spines['right'].set_visible(True)
            ax1.tick_params(labelsize=fontsize)
            ax1.yaxis.tick_right()

        if i != len(epis) - 1:
            ax1.set_xticks([])
            ax1.set_xticklabels([])
        # ax1.axis('off')
        # ax1.xaxis.set_visible(True)
        # plt.setp(ax1.spines.values(), visible=False)
        # ax1.yaxis.set_visible(True)

        ax1.set_xlim([-0.5, N - 0.5])
        if epi_labels:
            ax1.set_ylabel(epi_labels[i], fontsize=fontsize, rotation=0)
    ax1.spines['bottom'].set_visible(True)
    if x_ticks:
        tick_pos = np.linspace(0, N - 1, len(x_ticks))  # 这个坐标其实是不对的 差1个bin 但是为了ticks好看只有先这样了
        ax1.set_xticks(tick_pos)
        ax1.set_xticklabels(x_ticks, fontsize=fontsize)
    else:
        ax1.set_xticks([])
        ax1.set_xticklabels([])

    plt.savefig(output)
    plt.close()

In [29]:
visualize_HiC_epigenetics(mat.toarray(), tf.transpose(hms), f"./{cell_line}/ground_truth_chr{chr_num}_hic_vis.png",
                          colorbar=True, interval_after_heatmap=0.,
                          interval_between_epi=0., epi_labels=["H3K27me3", "H3K36me3", "H3K4me1", "H3K4me3", "H3K9me3"],
                          epi_yaxis=True, fontsize=20, epi_ratio=0.045,
                          x_ticks=['', '', '', '', '', ''], vmax=np.quantile(mat.toarray(), 0.98))

In [30]:
pred_hic = np.zeros(shape = mat.shape)

slice_len = 256
num_bins = len(hms)
diag_ind = 0

for i in range(0, (num_bins // slice_len) + 1):
  if (i+1) * slice_len > num_bins:
    i = num_bins / slice_len - 1
  hms_sub = np.expand_dims(hms[int(i*slice_len): int((i+1) * slice_len)].numpy(), axis=0)
  for j in range(0, (num_bins // slice_len) + 1):
    if (j+1) * slice_len > num_bins:
      j = num_bins / slice_len - 1
    hic_sub = np.expand_dims(mat[int(i*slice_len):int((i+1)*slice_len), int(j*slice_len):int((j+1)*slice_len)].toarray(), axis=0)
    hi_hic = autoencoder((hic_sub, hms_sub), training=False)
    pred_hic[int(i*slice_len):int((i+1)*slice_len), int(j*slice_len):int((j+1)*slice_len)] = np.squeeze(hi_hic.numpy())


visualize_HiC_epigenetics(pred_hic, tf.transpose(hms), f"./{cell_line}/pred_chr{chr_num}_hic_vis.png",
                          colorbar=True, interval_after_heatmap=0.,
                          interval_between_epi=0., epi_labels=["H3K27me3", "H3K36me3", "H3K4me1", "H3K4me3", "H3K9me3"],
                          epi_yaxis=True, fontsize=20, epi_ratio=0.045,
                          x_ticks=['', '', '', '', '', ''], vmax=np.quantile(pred_hic, 0.98))